In [19]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

In [ ]:
CONFIG = {
    "overwrite_intensity_ratio": 0.5, 
    "overwrite_dark_count_base": 100, 
    "overwrite_dark_count_ratio": 0.15, 
    "pen_intensity_std": 25,           
    "color_diff_threshold": 25,        
    "stroke_std_threshold": 5,         
    "texture_diff_threshold": 1.0,     
    "pressure_variance_threshold": 700,
    "edge_sharpness_threshold": 15.0,  
    "analyses_to_run": ["overwrite", "pen_variation", "color", "stroke", "texture", "pressure", "edge"],
    "confidence_threshold": 0.3,       
}

In [ ]:
def preprocess_image(image_path):
    img_color = cv2.imread(image_path)
    if img_color is None:
        raise ValueError("Image not found or unable to load")
    img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.GaussianBlur(img_gray, (5, 5), 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_gray = clahe.apply(img_gray)
    img_normalized = cv2.normalize(img_gray, None, 0, 255, cv2.NORM_MINMAX)
    return img_color, img_normalized

In [ ]:
def find_text_regions(img, img_color_debug=None):
    _, img_bin = cv2.threshold(img, 180, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((5, 5), np.uint8)
    img_bin = cv2.dilate(img_bin, kernel, iterations=1)

    contours, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    regions = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 10 and h > 10:  
            regions.append((x, y, w, h))

    merged_regions = []
    while regions:
        x1, y1, w1, h1 = regions.pop(0)
        merged = False
        for i in range(len(regions) - 1, -1, -1):
            x2, y2, w2, h2 = regions[i]
            if (abs(x1 - x2) < max(w1, w2) / 2 and abs(y1 - y2) < max(h1, h2) / 2):
                x = min(x1, x2)
                y = min(y1, y2)
                w = max(x1 + w1, x2 + w2) - x
                h = max(y1 + h1, y2 + h2) - y
                regions.pop(i)
                regions.append((x, y, w, h))
                merged = True
                break
        if not merged:
            merged_regions.append((x1, y1, w1, h1))

    merged_regions.sort(key=lambda r: r[0])
    if img_color_debug is not None:
        debug_img = img_color_debug.copy()
        for i, (x, y, w, h) in enumerate(merged_regions):
            cv2.rectangle(debug_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(debug_img, f"R{i}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        cv2.imwrite("debug_regions.png", debug_img)

    return merged_regions, ["Unknown" for _ in merged_regions]

In [ ]:
def detect_overwriting(img, regions):
    intensities = []
    dark_counts = []
    dark_thresholds = []
    for (x, y, w, h) in regions:
        region = img[y:y+h, x:x+w]
        text_pixels = region[region < 200]
        avg_intensity = np.mean(text_pixels) if text_pixels.size > 0 else 255
        dark_pixels = np.sum(region < (avg_intensity * 0.7))
        area = w * h
        dark_threshold = max(CONFIG["overwrite_dark_count_base"], area * CONFIG["overwrite_dark_count_ratio"])
        intensities.append(avg_intensity)
        dark_counts.append(dark_pixels)
        dark_thresholds.append(dark_threshold)

    avg_intensity_all = np.mean(intensities) if intensities else 255
    suspicious_regions = []
    confidence_scores = []
    for i, (intensity, dark_count, dark_threshold) in enumerate(zip(intensities, dark_counts, dark_thresholds)):
        intensity_score = max(0, (avg_intensity_all * CONFIG["overwrite_intensity_ratio"] - intensity) / (avg_intensity_all * CONFIG["overwrite_intensity_ratio"]))
        dark_score = max(0, (dark_count - dark_threshold) / dark_threshold)
        confidence = min(1, max(intensity_score, dark_score))
        if intensity < avg_intensity_all * CONFIG["overwrite_intensity_ratio"] or dark_count > dark_threshold:
            suspicious_regions.append(i)
            confidence_scores.append(confidence)

    print(f"Debug - Overwriting: Intensities: {intensities}, Dark Counts: {dark_counts}, Dark Thresholds: {dark_thresholds}")
    return suspicious_regions, intensities, dark_counts, confidence_scores, dark_thresholds

In [ ]:
def detect_pen_variation(intensities):
    if len(intensities) < 2:
        return False, 0
    intensity_std = np.std(intensities)
    confidence = min(1, (intensity_std - CONFIG["pen_intensity_std"]) / CONFIG["pen_intensity_std"])
    print(f"Debug - Pen Variation: Intensity Std: {intensity_std}")
    return intensity_std > CONFIG["pen_intensity_std"], max(0, confidence)

In [ ]:
def detect_color_difference(img_color, regions):
    if len(regions) < 2:
        return False, [], [], [], []

    avg_colors = []
    for (x, y, w, h) in regions:
        region = img_color[y:y+h, x:x+w]
        gray_region = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
        mask = gray_region < 200
        if not np.any(mask):
            avg_colors.append([0, 0, 0])
            continue
        b = np.mean(region[mask, 0])
        g = np.mean(region[mask, 1])
        r = np.mean(region[mask, 2])
        avg_colors.append([r, g, b])

    color_diffs = []
    for i in range(len(avg_colors) - 1):
        color1 = np.array(avg_colors[i])
        color2 = np.array(avg_colors[i + 1])
        diff = np.linalg.norm(color1 - color2)
        color_diffs.append(diff)

    suspicious_color_regions = []
    confidence_scores = []
    for i, diff in enumerate(color_diffs):
        confidence = min(1, (diff - CONFIG["color_diff_threshold"]) / CONFIG["color_diff_threshold"])
        if diff > CONFIG["color_diff_threshold"]:
            suspicious_color_regions.append(i)
            confidence_scores.append(confidence)

    print(f"Debug - Color Difference: Color Diffs: {color_diffs}")
    return len(suspicious_color_regions) > 0, suspicious_color_regions, avg_colors, color_diffs, confidence_scores

In [ ]:
def compute_stroke_width(img, regions):
    stroke_widths = []
    for (x, y, w, h) in regions:
        region = img[y:y+h, x:x+w]
        _, binary = cv2.threshold(region, 200, 255, cv2.THRESH_BINARY_INV)
        dist = cv2.distanceTransform(binary, cv2.DIST_L2, 3)
        stroke_width = np.mean(dist[binary == 255])
        stroke_widths.append(stroke_width if not np.isnan(stroke_width) else 0)
    return stroke_widths

In [ ]:
def detect_stroke_variation(stroke_widths):
    if len(stroke_widths) < 2:
        return False, 0
    stroke_std = np.std(stroke_widths)
    confidence = min(1, (stroke_std - CONFIG["stroke_std_threshold"]) / CONFIG["stroke_std_threshold"])
    print(f"Debug - Stroke Variation: Stroke Widths: {stroke_widths}, Std: {stroke_std}")
    return stroke_std > CONFIG["stroke_std_threshold"], max(0, confidence)

In [ ]:
def compute_texture(img, regions):
    textures = []
    for (x, y, w, h) in regions:
        region = img[y:y+h, x:x+w]
        lbp = local_binary_pattern(region, P=8, R=1, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), density=True)
        textures.append(hist)
    return textures

In [ ]:
def detect_texture_variation(textures):
    if len(textures) < 2:
        return False, 0
    diffs = []
    for i in range(len(textures) - 1):
        diff = np.sum(np.abs(textures[i] - textures[i + 1]))
        diffs.append(diff)
    max_diff = max(diffs) if diffs else 0
    confidence = min(1, (max_diff - CONFIG["texture_diff_threshold"]) / CONFIG["texture_diff_threshold"])
    print(f"Debug - Texture Variation: Texture Diffs: {diffs}")
    return max_diff > CONFIG["texture_diff_threshold"], max(0, confidence)

In [ ]:
def detect_pressure_variation(img, regions):
    variances = []
    for (x, y, w, h) in regions:
        region = img[y:y+h, x:x+w]
        text_pixels = region[region < 200]
        variance = np.var(text_pixels) if text_pixels.size > 0 else 0
        variances.append(variance)

    if len(variances) < 2:
        return False, 0

    variance_diff = max(variances) - min(variances)
    if variance_diff < 100:  # Increased to reduce false positives
        return False, 0

    variance_std = np.std(variances)
    confidence = min(1, (variance_std - CONFIG["pressure_variance_threshold"]) / CONFIG["pressure_variance_threshold"])
    print(f"Debug - Pressure Variation: Variances: {variances}, Std: {variance_std}")
    return variance_std > CONFIG["pressure_variance_threshold"], max(0, confidence)

In [ ]:
def compute_edge_sharpness(img, regions):
    sharpness_scores = []
    for (x, y, w, h) in regions:
        region = img[y:y+h, x:x+w]
        sobelx = cv2.Sobel(region, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(region, cv2.CV_64F, 0, 1, ksize=3)
        gradient_mag = np.sqrt(sobelx**2 + sobely**2)
        sharpness = np.mean(gradient_mag)
        sharpness_scores.append(sharpness if not np.isnan(sharpness) else 0)
    return sharpness_scores

In [ ]:
def detect_edge_sharpness_variation(sharpness_scores):
    if len(sharpness_scores) < 2:
        return False, 0

    sharpness_diff = max(sharpness_scores) - min(sharpness_scores)
    if sharpness_diff < 35:  
        return False, 0

    sharpness_std = np.std(sharpness_scores)
    confidence = min(1, (sharpness_std - CONFIG["edge_sharpness_threshold"]) / (2 * CONFIG["edge_sharpness_threshold"]))
    print(f"Debug - Edge Sharpness Variation: Sharpness Scores: {sharpness_scores}, Std: {sharpness_std}")
    return sharpness_std > CONFIG["edge_sharpness_threshold"], max(0, confidence)

In [ ]:
def is_fraudulent(img_color, img_gray):
    regions, recognized_text = find_text_regions(img_gray, img_color)
    if not regions:
        print("No text detected")
        return "No text detected", 0

    suspicious_regions, intensities, dark_counts, overwrite_confidence, dark_thresholds = detect_overwriting(img_gray, regions)
    pen_variation, pen_confidence = detect_pen_variation(intensities)
    color_variation, suspicious_color_regions, avg_colors, color_diffs, color_confidence = detect_color_difference(img_color, regions)

    stroke_widths = compute_stroke_width(img_gray, regions)
    stroke_variation, stroke_confidence = detect_stroke_variation(stroke_widths)

    textures = compute_texture(img_gray, regions)
    texture_variation, texture_confidence = detect_texture_variation(textures)

    pressure_variation, pressure_confidence = detect_pressure_variation(img_gray, regions)

    edge_sharpness_scores = compute_edge_sharpness(img_gray, regions)
    edge_variation, edge_confidence = detect_edge_sharpness_variation(edge_sharpness_scores)

    confidences = {}
    if suspicious_regions:
        confidences["overwrite"] = max(overwrite_confidence) if overwrite_confidence else 0
    if pen_variation:
        confidences["pen_variation"] = pen_confidence
    if color_variation:
        confidences["color"] = max(color_confidence) if color_confidence else 0
    if stroke_variation:
        confidences["stroke"] = stroke_confidence
    if texture_variation:
        confidences["texture"] = texture_confidence
    if pressure_variation:
        confidences["pressure"] = pressure_confidence
    if edge_variation:
        confidences["edge"] = edge_confidence

    overall_confidence = max(confidences.values()) if confidences else 0

    print("=== Fraud Detection Report ===")
    print(f"Regions (x, y, w, h): {regions}")

    # Fraud detection result
    result = "Normal"
    if suspicious_regions or pen_variation or color_variation or stroke_variation or texture_variation or pressure_variation or edge_variation:
        result = "Suspicious: "
        if suspicious_regions:
            result += f"Possible overwriting in regions {suspicious_regions} (confidence: {max(overwrite_confidence):.2f}) "
        if pen_variation:
            result += f"Possible different pens (intensity variation, confidence: {pen_confidence:.2f}) "
        if color_variation:
            result += f"Possible different pens (color variation in regions {suspicious_color_regions}, confidence: {max(color_confidence):.2f}) "
        if stroke_variation:
            result += f"Possible different pens (stroke width variation, confidence: {stroke_confidence:.2f}) "
        if texture_variation:
            result += f"Possible different pens (texture variation, confidence: {texture_confidence:.2f}) "
        if pressure_variation:
            result += f"Possible different pressure (variance in regions, confidence: {pressure_confidence:.2f}) "
        if edge_variation:
            result += f"Possible different pens (edge sharpness variation, confidence: {edge_confidence:.2f}) "

    # Apply probability threshold
    if overall_confidence < CONFIG["confidence_threshold"]:
        result = "Normal"
        overall_confidence = 0

    print(f"Result: {result}")
    print(f"Overall Confidence: {overall_confidence:.2f}")

    return result, overall_confidence


In [27]:
if __name__ == "__main__":
    try:
        image_path = r"C:\Users\hssin\Downloads\fraud chaker.png"
        img_color, img_gray = preprocess_image(image_path)
        result, confidence = is_fraudulent(img_color, img_gray)
    except Exception as e:
        print(f"Error: {str(e)}")

Debug - Overwriting: Intensities: [99.81037151702786, 120.76657060518733, 146.884778012685, 142.50485436893203, 150.680157946693], Dark Counts: [453, 273, 134, 151, 109], Dark Thresholds: [398.25, 270.0, 512.4, 351.0, 318.0]
Debug - Pen Variation: Intensity Std: 19.199663267777787
Debug - Color Difference: Color Diffs: [25.717129409509557, 51.91520044260745, 4.485806475213348, 15.375029872268742]
Debug - Stroke Variation: Stroke Widths: [2.9549968, 2.9220293, 1.8277824, 2.1848397, 2.1500223], Std: 0.45087406039237976
Debug - Texture Variation: Texture Diffs: [0.13299435028248585, 0.32450169138693735, 0.10213174803338734, 0.19693597806805357]
Debug - Pressure Variation: Variances: [3508.0375704502108, 2433.2759622065905, 1399.0490917583327, 1759.1140541050052, 1073.220504614737], Std: 864.1792378917295
Debug - Edge Sharpness Variation: Sharpness Scores: [163.63921900877412, 171.43668606311346, 109.88034487291438, 128.6412412998971, 135.9756634667256], Std: 22.72073571196387
=== Fraud De